In [33]:
from pympi.Elan import Eaf
from src.settings import ORKPJM_ANN_DIR
from loguru import logger
from collections import defaultdict
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

## Tiers:

### Data structure
_TimeStamp: int
> - in `TIME_UNITS`, miliseconds

_Content: str
> - `glosa*`: utf-8,
> - `HamNoSys`: unicode (font HamNoSys)
> - `tlumaczenie_publikacja`: utf-8

Tier results:
```python
List[Tuple[ts1: _TimeStamp, ts2: _TimeStamp, content: _Content]]
```

In [48]:
tier_records = defaultdict(list)
for task_dir_path in tqdm(list(ORKPJM_ANN_DIR.iterdir()), desc="Tasks"):
    if not task_dir_path.is_dir():
        continue
    task_num = task_dir_path.name
    for eaf_doc_filepath in task_dir_path.glob("*.eaf"):
        eac_doc_rel_filepath = eaf_doc_filepath.as_posix().replace(ORKPJM_ANN_DIR.as_posix(), "")
        eaf_doc = Eaf(eaf_doc_filepath)
        if len(eaf_doc.media_descriptors) > 1:
            logger.warning(f"More than one media descriptors! Getting the first one...\n{eaf_doc.media_descriptors=}")
        media_descriptor = eaf_doc.media_descriptors[0]
        vid_fname = media_descriptor["MEDIA_URL"]
        time_units = eaf_doc.header["TIME_UNITS"]
        for tier_name in eaf_doc.get_tier_names():
            tier_data = eaf_doc.get_annotation_data_for_tier(tier_name)
            for tier_record in tier_data:
                record = {
                    "TaskNum": task_num,
                    "DocFilePath": eac_doc_rel_filepath,
                    "VideoFileName": vid_fname,
                    "TimeUnit": time_units,
                    "TierId": tier_name,
                    "TimeStampStart": tier_record[0],
                    "TimeStampEnd": tier_record[1],
                    "Content": tier_record[2],
                }
                tier_records[tier_name].append(record)

Tasks:   0%|          | 0/7 [00:00<?, ?it/s]

In [49]:
glosa_df = pd.DataFrame.from_records(tier_records["glosa"])
glosa_df

,TaskNum,DocFilePath,VideoFileName,TimeUnit,TierId,TimeStampStart,TimeStampEnd,Content
0,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,glosa,29960,31200,%
1,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,glosa,38560,39040,%
2,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,glosa,39040,39440,MYŚLEĆ 2.1 P:Z;L:Z (NA PRZEMIAN)
3,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,glosa,39440,39640,JAK 1.2 P:I;L:Ø
4,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,glosa,39640,39920,POMYSŁ 1.3 P:AZ;L:Ø
...,...,...,...,...,...,...,...,...
160671,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,glosa,353480,354480,NIEDŹWIEDŹ/MIŚ 1.2 P:B;L:Ø
160672,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,glosa,354680,356440,PRZESTRASZYĆ 1.1 P:5A;L:5A (JAK ZŁAPAĆ)
160673,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,glosa,359240,360120,BIUSTONOSZ 1.2. P:5;L:5 (STANIK/STRÓJ KĄPIELOWY)
160674,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,glosa,360120,360960,DZIECKO 2.2 P:I1;LØ


In [56]:
tlumaczenie_df = pd.DataFrame.from_records(tier_records["tlumaczenie_publikacja"])
tlumaczenie_df

,TaskNum,DocFilePath,VideoFileName,TimeUnit,TierId,TimeStampStart,TimeStampEnd,Content
0,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,tlumaczenie_publikacja,39040,41120,"Myślę, że mam inny pomysł, można?"
1,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,tlumaczenie_publikacja,41120,49680,"Chyba ten znak mówi, że jak ktoś będzie spacer..."
2,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,tlumaczenie_publikacja,49680,55280,Trzeba przejść łukiem obok leżącego i o tym zn...
3,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,tlumaczenie_publikacja,61280,66840,"Okrągły znak pomaga nam, mówi, że są pasy na u..."
4,15,/15/K66BF13-26_15_15_signsNO.eaf,K66BF13-26.mp4,milliseconds,tlumaczenie_publikacja,66840,73200,"Jak ktoś zobaczy, ale zignoruje ten znak, to m..."
...,...,...,...,...,...,...,...,...
40350,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,tlumaczenie_publikacja,248960,250320,[uderzanie w coś]
40351,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,tlumaczenie_publikacja,279560,291400,Kot zobaczył w akwarium rybkę. Podszedł i dał ...
40352,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,tlumaczenie_publikacja,291400,347120,"Zjadłam, zjadłam."
40353,13,/13/K17BF13-26_13_13_comics.eaf,K17BF13-26.mp4,milliseconds,tlumaczenie_publikacja,347120,356440,"Zając biegnie, zobaczył wiszące pranie. Wskocz..."
